# **0.Librairies**

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
# Modèles
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Imbalanced-learn
from imblearn.over_sampling import SMOTE

# XGBoost et LightGBM
import xgboost as xgb
import lightgbm as lgb

# PyTorch 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# UCI ML Repo
from ucimlrepo import fetch_ucirepo

# **I. Formulation du problème**

### **1. Définition du problème**

Nous allons utiliser le dataset [Online Shoppers Purchasing Intention](https://archive.ics.uci.edu/dataset/468/online+shoppers+purchasing+intention+dataset) pour prédire si une session d'un utilisateur aboutira à un achat ou non. Il s'agit donc d'un problème de **classification binaire**.

- **Objectif :** Construire un modèle prédictif capable de classer les sessions en deux catégories :
    - **Revenue = True** : la session se termine par un achat.
    - **Revenue = False** : la session ne se termine pas par un achat.

### **2. Formulation mathématique**

Soit :
- Un ensemble de données $X=\{x_1,x_2,...,x_n\}$ où chaque xi est un vecteur de caractéristiques représentant une session. 
- Une variable cible $Y=\{y_1, y_2, ..., y_n\}$ où $y_i \in \{0, 1\}$ (0 pour "pas d'achat", 1 pour "achat").

Le but est de trouver une fonction $f : X \rightarrow Y$ telle que pour un nouvel exemple $x, f(x)$ prédit correctement $y$.

# **II. Exploration du dataset**

In [2]:
# Importation des données 
online_shoppers_purchasing_intention_dataset = fetch_ucirepo(id=468) 
  
# Récupérations des variables
variables = online_shoppers_purchasing_intention_dataset.variables 
X = online_shoppers_purchasing_intention_dataset.data.features 
y = online_shoppers_purchasing_intention_dataset.data.targets 

In [3]:
# Sauvegarde des données
variables.to_csv('../data/online_shoppers_purchasing_intention_dataset.csv', index=False)
X.to_csv('../data/features.csv', index=False)
y.to_csv('../data/targets.csv', index=False)

# Chargement des données
variables = pd.read_csv('../data/online_shoppers_purchasing_intention_dataset.csv')
X = pd.read_csv('../data/features.csv')
y = pd.read_csv('../data/targets.csv')

## **1. Analyse exploratoire des données**

Cette partie est dédiée à la **compréhension générale des variables** et consistera en l'étude des:
- **Variables numériques :** Analyse des distributions (moyenne, médiane, écart-type).
- **Variables catégorielles :** Comptage des occurrences, encodage.

*Selon la documentation, il n'y a aucune variable `Null`.*

### **a. Variables Dataset**

In [ ]:
variables.info()

In [5]:
# On supproime les variables vides
variables = variables[['name', 'role', 'type', 'missing_values']]

In [ ]:
variables

### **b. Target: Revenue (y)**

In [ ]:
y.info()

In [ ]:
y.describe()

In [ ]:
print(y['Revenue'].sum()/len(y))

# On affiche la distribution de la variable cible
sns.countplot(x='Revenue', data=y)
plt.show()

On observe que seulement **14,5%** des utilisateurs ont acheté.

### **c. Features (X)**

#### Informations générales

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
X.describe()

## **2. Visualisations initiales**

#### Visualisation des features **numériques**

In [ ]:
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
n = len(numeric_features)
cols = 4
rows = n // cols + (n % cols > 0)

# Créer la figure et les sous-graphes
fig, axes = plt.subplots(rows, cols, figsize=(16, 12))
axes = axes.flatten()
for i, feature in enumerate(numeric_features):
    sns.histplot(X[feature], kde=True, ax=axes[i])
    axes[i].set_title(f'Distribution de {feature}')
for i in range(n, len(axes)):
    fig.delaxes(axes[i])
plt.tight_layout()
plt.show()

- La majorité des distributions ont une forte concentration de données **proches de zéro**, particulièrement pour les interactions avec des pages spécifiques du site (pages administratives, informatives et produits). On peut supposer que que seulement une petite fraction des utilisateurs s'engagent sur ce types de pages.

- L'analyse des taux de sortie et de rebond pourrait aider à **identifier des problèmes d'engagement**, notamment si certains types de pages (comme les pages de produits ou informatives) ont des taux de sortie plus élevés.

In [ ]:
categorical_features = X.select_dtypes(include=['object', 'bool']).columns.tolist()
n = len(categorical_features)
cols = 3

# Créer la figure et les sous-graphes
fig, axes = plt.subplots(1, cols, figsize=(16, 4))
axes = axes.flatten()
for i, feature in enumerate(categorical_features):
    sns.histplot(X[feature], kde=True, ax=axes[i])
    axes[i].set_title(f'Distribution de {feature}')
for i in range(n, len(axes)):
    fig.delaxes(axes[i])
plt.tight_layout()
plt.show()

On remarque plusieurs choses:
- Un comportement de **saisonalité**, avec des pics au printemps et en automne.
  
- Une présence plus importante de **visiteurs réccurents**, cela indique que le site a un bon taux de fidélisation.
  
- Les achats semblent assez **équitablement répartis** au cours de la semaine, on retrouve une proportion cohérente d'environ 5/7 en semaine et 2/7 le week-end. Cela pourrait indiquer que le site est autant utilisé pour des activités professionnelles que récréatives.

Aucune variable nulle dans les **features** et le **target** dataset, nous pouvons donc directement travailler sur les données.

#### **Détection de corrélations :**
- Matrice de corrélation pour identifier les variables fortement liées.

In [ ]:
plt.figure(figsize=(12, 10))
correlation = X[numeric_features].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Matrice de corrélation')
plt.show()

**Corrélations fortes :**

- **ProductRelated** et **ProductRelated_Duration** (0.86) : Il est logique que le nombre de pages de produits visitées soit fortement corrélé avec la durée passée sur ces pages. Plus les utilisateurs visitent de pages produits, plus ils passent de temps sur ces pages.

- **BounceRates** et **ExitRates** (0.91) : Ces deux variables sont également très corrélées. Cela indique que les sessions avec un taux de rebond élevé ont tendance à se terminer rapidement (forte probabilité de quitter le site après avoir consulté peu de pages).

**Corrélations modérées :**

- **Informational** et **Informational_Duration** (0.62) : De même, une corrélation assez élevée entre le nombre de pages informatives visitées et la durée passée sur ces pages, ce qui est également intuitif.

- **Administrative** et **Administrative_Duration** (0.6) : La relation entre le nombre de pages administratives visitées et la durée passée sur ces pages est modérée. Plus les utilisateurs interagissent avec des pages administratives, plus ils passent de temps sur ces pages.

In [ ]:
data = pd.concat([X, y], axis=1)

plt.figure(figsize=(16, 12))
for i, feature in enumerate(numeric_features, 1):
    plt.subplot(4, 4, i)
    sns.violinplot(x='Revenue', y=feature, data=data, split=True)
    plt.title(f'{feature} vs Revenue')

plt.tight_layout()
plt.show()

- **Pages produits** : Les variables liées aux pages produits (*ProductRelated* et *ProductRelated_Duration*) sont les meilleures variables pour différencier les utilisateurs générant des revenus de ceux qui n'en génèrent pas. Plus un utilisateur interagit avec les pages produits, plus il est susceptible de générer un revenu.

- **Taux de rebond** et **taux de sortie** : Les utilisateurs qui génèrent des revenus tendent à avoir des taux de rebond et taux de sortie plus faibles, ce qui indique un engagement plus profond avec le site.

- **PageValues** : C’est l'une des variables les plus discriminantes, les utilisateurs générant des revenus ayant des valeurs de page beaucoup plus élevées.

- Les autres facteurs n'ont pas un impact significatif sur la génération de revenus, du moins dans cette visualisation.

# **III. Modélisation**

## **1. Prétraitement des données**

### **a. Encodage des variables catégorielles**

Convertir les variables binaires en entiers :

In [17]:
data['Weekend'] = data['Weekend'].astype(int)
data['Revenue'] = data['Revenue'].astype(int)


One-Hot Encoding pour les variables catégorielles avec plus de deux catégories :

In [18]:
data = pd.get_dummies(data, columns=['Month', 'VisitorType'])

In [ ]:
data.head()

### **b. Normalisation ou standardisation des variables numériques**
Standardisation pour uniformiser l'échelle des variables :

In [20]:
scaler = StandardScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])

### **c. Gestion du déséquilibre des classes**
SMOTE pour sur-échantillonner la classe minoritaire :

In [21]:
X = data.drop('Revenue', axis=1)
y = data['Revenue']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

### **d. Division des données en ensembles d'entraînement et de test**
Séparation les données en ensembles d'entraînement et de test :

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, 
    y_resampled, 
    test_size=0.3, # 30% des données pour le test
    random_state=42, # Fixer le random_state pour obtenir les mêmes résultats
    stratify=y_resampled # Stratification pour conserver la distribution de la variable cible
    )

## **2. Algorithmes de Classification**

Nous allons utliser les méthodes suivantes classiques puis comparer leurs performances:
- Régression Logistique
- Arbre de Décision
- Forêt Aléatoire
- XGBoost
- Réseau de Neurones avec PyTorch

### **a. Implémentation des modèles**

#### Régression Logistique

In [23]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

#### Arbre de Décision

In [24]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

#### Forêt Aléatoire

In [25]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

#### XGBoost

In [26]:
xgb_model = xgb.XGBClassifier(eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

#### RN avec PyTorch

In [27]:
# Convertir les données en tenseurs
X_train_tensor = torch.tensor(X_train.values.astype(float), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values.astype(float), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Créer des DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [28]:
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.layer2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 2)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.relu(self.layer1(x))
        out = self.relu(self.layer2(out))
        out = self.output(out)
        return out

model = NeuralNet(input_size=X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    all_preds = []
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_preds.extend(predicted.numpy())
    
    print(f'Accuracy du réseau sur les données de test : {100 * correct / total}%')

### **b. Évaluation des modèles**

La précision seule perd en fiabilité dans un scénario où les classes sont déséquilibrées. Nous allons donc utiliser 3 méthodes d'évaluation qui sont particulirement pertinentes dans ce cas, et importées de `sklearn.metrics`: 

- `classification_report`:
  - la **précision** du modèle (proportion des prédictions positives qui sont correctes).
  - le **recall** (proportion des vrais positifs qui sont correctement prédits).
  - le **f1-score** (moyenne harmonique de la précision et du rappel).

- `confusion_matrix`: affiche au sein d'une matrice 2x2 les **True Positives**, **True Negatives**, **False Positives** et **False Negatives**.

- `roc_auc_score`: **ROC** (Receiver Operating Characteristic) est le tracé de **True Positive Ratio** contre **False Positive Ratio** pour chaque seuil de classification possible entre 0 et 1 (dans la pratique, à des intervalles choisis). L'**AUC** est l'aire sous la courbe ROC, plus il est proche de 1, plus le modèle est performant, et s'il est proche de 0.5, cela équivaut à une classification aléatoire.

In [31]:
predictions = {
    'Régression Logistique': y_pred_lr,
    'Arbre de Décision': y_pred_dt,
    'Forêt Aléatoire': y_pred_rf,
    'XGBoost': y_pred_xgb,
    'Réseau de Neurones (PyTorch)': all_preds
}

#### **Classification Report**

In [ ]:
results = []

for model_name, y_pred in predictions.items():
    # Récupération des informations du rapport de classification sous forme de dictionnaire
    report = classification_report(y_test, y_pred, output_dict=True)
    precision_0 = report['0']['precision']
    precision_1 = report['1']['precision']
    recall_0 = report['0']['recall']
    recall_1 = report['1']['recall']
    f1_0 = report['0']['f1-score']
    f1_1 = report['1']['f1-score']

    results.append({
        'Modèle': model_name,
        'Précision False': precision_0,
        'Précision True': precision_1,
        'Recall False': recall_0,
        'Recall True': recall_1,
        'F1-Score False': f1_0,
        'F1-Score True': f1_1
    })

df_results = pd.DataFrame(results)
df_results

La **forêt aléatoire** semble obtenir les meilleurs résultats.

#### **Confusion Matrix**

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for i, (model_name, y_pred) in enumerate(predictions.items()):
    ax = axes[i//3, i%3] 
    cm = confusion_matrix(y_test, y_pred)  # Calcul de la matrice de confusion
    sns.heatmap(cm, annot=True, fmt='d', cmap='plasma', ax=ax)
    ax.set_title(f'Matrice de Confusion - {model_name}')
    ax.set_xlabel('Prédictions')
    ax.set_ylabel('Vérités terrain')

axes[1][2].axis('off')
plt.tight_layout()
plt.show()


La **forêt aléatoire** et **XGBoost** semblent légèrement meilleurs en termes de réduction des faux positifs et faux négatifs.

#### **ROC-AUC Score**

Afin de tracer la courbe **ROC** à différents seuils de classification, on calcule les probabilités d'appartenance à chaque classe pour chaque observation. En pratique, seule la probabilité $P(classe=1)$ nous intéresse. Pour obtenir ces probabilités, nous pouvons utiliser la méthode `predict_proba` de `sklearn`. Cependant, notre **RN** ne dispose pas de cette méthode, nous allons donc calculer les **logits** puis les probabilités via la fonction **sigmoid**.

In [ ]:
X_test_tensor = torch.FloatTensor(X_test.values.astype(float))
with torch.no_grad():
    logits = model(X_test_tensor) 
    probs_pytorch = F.sigmoid(logits).numpy()[:, 1]

probabilities = {
    'Régression Logistique': lr.predict_proba(X_test)[:, 1],
    'Arbre de Décision': dt.predict_proba(X_test)[:, 1],
    'Forêt Aléatoire': rf.predict_proba(X_test)[:, 1],
    'XGBoost': xgb_model.predict_proba(X_test)[:, 1],
    'Réseau de Neurones (PyTorch)': probs_pytorch
}

plt.figure(figsize=(10, 8))

for model_name, probs in probabilities.items():
    fpr, tpr, thresholds = roc_curve(y_test, probs)
    roc_auc = roc_auc_score(y_test, probs)
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Taux de Faux Positifs (FPR)')
plt.ylabel('Taux de Vrais Positifs (TPR)')
plt.title('Courbes ROC de plusieurs modèles')
plt.legend(loc='lower right')
plt.grid(True)

plt.show()

Ce graphique montre que les modèles **Forêt Aléatoire**, **XGBoost** et les **Réseaux de Neurones** surpassent les autres en termes de capacité à distinguer les classes, avec des AUC proches de 1.

# Conclusion

Le modèle **Forêt Aléatoire** nous apparaît comme étant le plus performant pour résoudre cette tâche de classification.